In [1]:
!pip install requests beautifulsoup4 pandas


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import re

base_url = "https://exclusive.vwgroupretail.es/coches/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

data = []

# Primero, averiguamos el número total de páginas
response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

page_total_tag = soup.find('span', class_='r-paginator__mobile-total')
if page_total_tag:
    total_pages = int(page_total_tag.get_text(strip=True))
else:
    # Si no encontramos el total de páginas, por simplicidad se asume 121 como en el ejemplo.
    total_pages = 121

for page in range(1, total_pages+1):
    url = f"{base_url}?page={page}"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraer datos del JSON-LD
        # Buscamos la etiqueta con el JSON-LD
        script_tag = soup.find('script', class_='yoast-schema-graph', type='application/ld+json')
        vehicle_dict = {}
        if script_tag:
            data_json = script_tag.get_text(strip=True)
            json_data = json.loads(data_json)

            # Navegamos hasta ItemList y luego los vehículos
            for item in json_data.get('@graph', []):
                if item.get('@type') == 'ItemList':
                    for vehicle_data in item.get('itemListElement', []):
                        vehicle = vehicle_data.get('item', {})
                        if vehicle.get('@type') == 'Vehicle':
                            # productId es un campo único que usaremos para unir con el HTML
                            product_id = vehicle.get('productId', 'N/A')
                            # Extraemos los datos del JSON
                            numberOfDoors = vehicle.get('numberOfDoors', 'N/A')
                            vehicleSeatingCapacity = vehicle.get('vehicleSeatingCapacity', 'N/A')
                            vehicleTransmission = vehicle.get('vehicleTransmission', 'N/A')
                            vehicleModelDate = vehicle.get('vehicleModelDate', 'N/A')
                            fuelType = vehicle.get('fuelType', 'N/A')
                            model = vehicle.get('model', 'N/A')
                            emissionsCO2 = vehicle.get('emissionsCO2', 'N/A')

                            offers = vehicle.get('offers', {})
                            priceCurrency = offers.get('priceCurrency', 'N/A')
                            price_ld = offers.get('price', 'N/A')
                            availability = offers.get('availability', 'N/A')
                            seller = offers.get('seller', {})
                            seller_name = seller.get('name', 'N/A')
                            seller_url = seller.get('url', 'N/A')

                            vehicle_dict[product_id] = {
                                'numberOfDoors': numberOfDoors,
                                'vehicleSeatingCapacity': vehicleSeatingCapacity,
                                'vehicleTransmission': vehicleTransmission,
                                'vehicleModelDate': vehicleModelDate,
                                'fuelType_json': fuelType,  # del JSON
                                'model': model,
                                'emissionsCO2': emissionsCO2,
                                'priceCurrency': priceCurrency,
                                'price_ld': price_ld,
                                'availability': availability,
                                'seller_name': seller_name,
                                'seller_url': seller_url
                            }

        # Extraemos los vehículos del HTML
        vehicles = soup.select('div.card.vcard')  
        if not vehicles:
            print(f"No se encontraron vehículos en la página {page}")
        
        for vehicle in vehicles:
            # Datos desde HTML
            title_tag = vehicle.find('a', class_='vcard--link')
            title = title_tag.get_text(strip=True) if title_tag else 'N/A'
            link = title_tag['href'] if title_tag else 'N/A'

            price_tag = vehicle.find('div', class_='vcard-price__price')
            price_html = price_tag.get_text(strip=True) if price_tag else 'N/A'
            
            info_tag = vehicle.find('p', class_='vcard-consumption__title')
            if info_tag:
                info_text = info_tag.get_text(strip=True)
                parts = info_text.split(' - ')
                if len(parts) == 3:
                    date, kms, fuel = parts
                else:
                    date, kms, fuel = 'N/A', 'N/A', 'N/A'
            else:
                date, kms, fuel = 'N/A', 'N/A', 'N/A'

            # Extraemos productId desde el link (la URL suele terminar con el productId/)
            # Por ejemplo: https://exclusive.vwgroupretail.es/coches/segunda-mano/.../960877/
            product_id_match = re.search(r'/(\d+)/?$', link)
            if product_id_match:
                product_id = product_id_match.group(1)
            else:
                product_id = 'N/A'

            # Datos del JSON para este vehículo
            json_info = vehicle_dict.get(product_id, {})
            
            data.append({
                'productId': product_id,
                'Title': title,
                'Link': link,
                'Date': date,
                'Kms': kms,
                'Fuel_HTML': fuel,  # desde el HTML
                'Price_HTML': price_html,

                # Datos del JSON
                'numberOfDoors': json_info.get('numberOfDoors', 'N/A'),
                'vehicleSeatingCapacity': json_info.get('vehicleSeatingCapacity', 'N/A'),
                'vehicleTransmission': json_info.get('vehicleTransmission', 'N/A'),
                'vehicleModelDate': json_info.get('vehicleModelDate', 'N/A'),
                'fuelType_JSON': json_info.get('fuelType_json', 'N/A'),
                'model': json_info.get('model', 'N/A'),
                'emissionsCO2': json_info.get('emissionsCO2', 'N/A'),
                'priceCurrency': json_info.get('priceCurrency', 'N/A'),
                'price_LD': json_info.get('price_ld', 'N/A'),
                'availability': json_info.get('availability', 'N/A'),
                'seller_name': json_info.get('seller_name', 'N/A'),
                'seller_url': json_info.get('seller_url', 'N/A')
            })

        print(f"Página {page} procesada correctamente")
        time.sleep(2)  # Evita solicitudes muy rápidas

    except Exception as e:
        print(f"Error en la página {page}: {e}")
        time.sleep(5)

df = pd.DataFrame(data)
df.to_csv('vehicles.csv', index=False)
print("Datos guardados en 'vehicles.csv'")



Página 1 procesada correctamente
Página 2 procesada correctamente
Página 3 procesada correctamente
Página 4 procesada correctamente
Página 5 procesada correctamente
Página 6 procesada correctamente
Página 7 procesada correctamente
Página 8 procesada correctamente
Página 9 procesada correctamente
Página 10 procesada correctamente
Página 11 procesada correctamente
Página 12 procesada correctamente
Página 13 procesada correctamente
Página 14 procesada correctamente
Página 15 procesada correctamente
Página 16 procesada correctamente
Página 17 procesada correctamente
Página 18 procesada correctamente
Página 19 procesada correctamente
Página 20 procesada correctamente
Página 21 procesada correctamente
Página 22 procesada correctamente
Página 23 procesada correctamente
Página 24 procesada correctamente
Página 25 procesada correctamente
Página 26 procesada correctamente
Página 27 procesada correctamente
Página 28 procesada correctamente
Página 29 procesada correctamente
Página 30 procesada cor

In [4]:
print(soup.prettify())  # Esto imprimirá el contenido HTML de la página


<!DOCTYPE html>
<html class="no-js" lang="es">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://exclusive.vwgroupretail.es/xmlrpc.php" rel="pingback"/>
  <script type="text/javascript">
   window.dataLayer = window.dataLayer || [];
            function gtag() { dataLayer.push(arguments); }
  </script>
  <script type="text/javascript">
   var iubenda_configuration_options = {"lang":"es","siteId":1783013,"cookiePolicyId":56337298,"consentOnScroll":false,"consentOnLinkAndButton":false,"perPurposeConsent":true,"consentOnElement":"","preferenceCookie":{"expireAfter":365},"banner":{"position":"bottom","content":"Este sitio web y las herramientas de terceros integradas en \u00e9l procesan datos personales (por e